In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/health-insurance-cross-sell-prediction/sample_submission.csv
/kaggle/input/health-insurance-cross-sell-prediction/train.csv
/kaggle/input/health-insurance-cross-sell-prediction/test.csv


In [3]:
train_df = pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction/train.csv')
test_df = pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction/test.csv')

In [4]:
train_df.columns

Index(['id', 'Gender', 'Age', 'Driving_License', 'Region_Code',
       'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium',
       'Policy_Sales_Channel', 'Vintage', 'Response'],
      dtype='object')

In [6]:
train_df

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0
...,...,...,...,...,...,...,...,...,...,...,...,...
381104,381105,Male,74,1,26.0,1,1-2 Year,No,30170.0,26.0,88,0
381105,381106,Male,30,1,37.0,1,< 1 Year,No,40016.0,152.0,131,0
381106,381107,Male,21,1,30.0,1,< 1 Year,No,35118.0,160.0,161,0
381107,381108,Female,68,1,14.0,0,> 2 Years,Yes,44617.0,124.0,74,0


In [11]:
train_df.isna().sum().sum() , test_df.isna().sum().sum()

(0, 0)

In [21]:
train_df['Response'].value_counts()

Response
0    334399
1     46710
Name: count, dtype: int64

In [12]:
train_df['Gender'].unique(), train_df['Vehicle_Age'].unique(), train_df['Vehicle_Damage'].unique()

(array(['Male', 'Female'], dtype=object),
 array(['> 2 Years', '1-2 Year', '< 1 Year'], dtype=object),
 array(['Yes', 'No'], dtype=object))

In [13]:
gender_mapping = {
    'Male' : 0,
    'Female': 1
}

vehicle_age_mapping = {
    '< 1 Year': 0,
    '1-2 Year': 1,
    '> 2 Years': 2
}

vehicle_damage_mapping = {
    'Yes': 1,
    'No' : 0
}
train_df['Gender'] = train_df['Gender'].map(gender_mapping)
train_df['Vehicle_Age'] = train_df['Vehicle_Age'].map(vehicle_age_mapping)
train_df['Vehicle_Damage'] = train_df['Vehicle_Damage'].map(vehicle_damage_mapping)


In [15]:
X = train_df.drop(columns = ['id','Response'])
y = train_df['Response']

from sklearn.model_selection import train_test_split

X_train , X_valid , y_train , y_valid = train_test_split(X,y , test_size = 0.2, random_state = 42)

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
class_weights = {0: 1, 1 : 2.5}
lr = LogisticRegression(class_weight= class_weights,max_iter = 500)

lr.fit(X_train , y_train)

y_pred = lr.predict(X_valid)
accuracy_score(y_pred ,y_valid)

0.823410563879195

In [37]:
print(classification_report(y_pred, y_valid))

              precision    recall  f1-score   support

           0       0.88      0.91      0.90     64237
           1       0.42      0.34      0.37     11985

    accuracy                           0.82     76222
   macro avg       0.65      0.63      0.64     76222
weighted avg       0.81      0.82      0.81     76222



In [38]:
np.unique(y_pred , return_counts= True) ,y_valid.value_counts()

((array([0, 1]), array([64237, 11985])),
 Response
 0    66699
 1     9523
 Name: count, dtype: int64)